In [ ]:
visit_source = '4.3'

# Yearly DDF cadence plots for one opsim

In [ ]:
import sys

sched_source = 'usdf'
#sched_source = 'shared'
#sched_source = 'devel'
match sched_source:
    case 'shared':
        sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/rubin_scheduler-3.7.0")
        sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/SP-2165/rubin_sim")
        sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/SP-2120/schedview")
    case 'devel':
        sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/rubin_scheduler")
        sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/rubin_sim")
        sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/schedview")
    case _:
        # Use whatever is in the kernel
        pass

In [ ]:
import datetime
import warnings
import schedview.collect
import schedview.plot
from rubin_sim.data import get_baseline
from lsst.resources import ResourcePath
from rubin_scheduler.utils import ddf_locations
from schedview import DayObs
from astropy.time import Time
import bokeh.io
import numpy as np
from erfa import ErfaWarning
from IPython.display import display, Markdown

In [ ]:
from schedview.collect.visits import OPSIMDB_TEMPLATE

In [ ]:
# In simulations, we go far enough into the future that the erfa module finds it "dubious".
# Keep the complaints quiet.
warnings.filterwarnings(
    "ignore",
    category=ErfaWarning,
    message=r".*dubious year.*",
)

In [ ]:
bokeh.io.output_notebook(hide_banner=True)

In [ ]:
opsim_fname = get_baseline() if visit_source=='baseline' else OPSIMDB_TEMPLATE.format(sim_version=visit_source)
visits_rp = ResourcePath(opsim_fname)
visits = schedview.collect.read_opsim(visits_rp, stackers=schedview.collect.visits.DDF_STACKERS)

In [ ]:
print(f"Read visits from {opsim_fname}")

In [ ]:
ddf_field_names = tuple(ddf_locations().keys())
# Different versions of the schedule include a DD: prefix, or not.
# Catch them all.
ddf_field_names = ddf_field_names + tuple([f"DD:{n}" for n in ddf_field_names])

# Figure out which column has the target names.
target_column_name = "target_name" if "target_name" in visits.columns else "target"
ddf_visits = visits.loc[visits[target_column_name].isin(ddf_field_names)]

In [ ]:
nightly_ddf = schedview.compute.visits.accum_teff_by_night(ddf_visits)
for year in np.arange(2025, 2036):
    start_mjd = int(Time(f"{year}-06-21").mjd)
    end_mjd = start_mjd + 365
    cadence_plots = schedview.plot.create_cadence_plot(nightly_ddf, start_mjd, end_mjd)
    display(Markdown(f"## DDF Cadence for {year}-{year+1} of opsim {visit_source}"))
    bokeh.io.show(cadence_plots)